In [1]:
import os
from src.customModel.customModel import TorchAttentionModel3, TorchAttentionModel4

from src.constant import SRC_PATH
os.environ["L5KIT_DATA_FOLDER"] = '/workspace/datasets'
# os.environ['CUDA_VISIBLE_DEVICES']= '0'
# os.environ["TUNE_RESULT_DIR"] =  '/DATA/l5kit/rllib_tb_logs'
import gym
from l5kit.configs import load_config_data
from l5kit.environment.envs.l5_env import SimulationConfigGym, GymStepOutput, L5Env
from l5kit.environment.envs.l5_env2 import SimulationConfigGym, GymStepOutput, L5Env2
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet, CLEMetricSet
from prettytable import PrettyTable
import datetime
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
import torch.nn as nn
import numpy as np
import gym
from typing import Dict
import numpy as np
import ray
import pytz
from ray import tune

In [2]:
ray.init(num_cpus=9, ignore_reinit_error=True, log_to_driver=False, object_store_memory = 5*10**9, local_mode=False)


from l5kit.configs import load_config_data

env_config_path = SRC_PATH + 'src/configs/gym_vectorizer_config.yaml'
env_config_path = SRC_PATH + 'src/configs/gym_vectorizer_config_hist3.yaml'
cfg = load_config_data(env_config_path)

2023-04-22 07:22:35,584	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67039232 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=5.12gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-04-22 07:22:36,641	INFO worker.py:1538 -- Started a local Ray instance.


In [3]:
from ray import tune
from src.customEnv.wrapper import L5EnvWrapper, L5EnvWrapperTorch
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}

tune.register_env("L5-CLE-V2", lambda config: L5Env2(**env_kwargs))
ModelCatalog.register_custom_model( "TorchAttentionModel3", TorchAttentionModel3)
ModelCatalog.register_custom_model( "TorchAttentionModel4", TorchAttentionModel4)

## Train

In [ ]:
import ray
from ray import air, tune
hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/home/pronton/ray_results/debug_vector_ppo_separated_kin_hist3' + date

train_envs = 4
lr = 3e-4
lr_start = 3e-5
lr_end = 3e-6
lr_time = int(4e6)

config_param_space = {
    "env": "L5-CLE-V2",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 8,
    "num_envs_per_worker": train_envs,
    "model": {
        "custom_model": "TorchAttentionModel4",
        "custom_model_config": {'cfg':cfg,
                                'freeze_actor': False,
                                'shared_feature_extractor': False,
                                },
    },
    
    # 'model' : {
    #         # "dim": 84,
    #         # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
    #         # "conv_activation": "relu",
    #         "post_fcnet_hiddens": [256],
    #         "post_fcnet_activation": "relu",
    #         "vf_share_layers": False,   
    # },
    
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    "lr": lr,
    'seed': 42,
    "lr_schedule": [
        [7e5, lr_start],
        [2e6, lr_end],
    ],
    'train_batch_size': 1024, # 8000 
    'sgd_minibatch_size': 512, #2048
    'num_sgd_iter': 10,#16,
    'batch_mode': 'truncate_episodes',
    # "rollout_fragment_length": 32,
    'gamma': 0.8,    
}

result_grid = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(6e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=2, 
                                               checkpoint_frequency = 10, 
                                               checkpoint_score_attribute = 'episode_reward_mean'),
        # callbacks=[WandbLoggerCallback(project="l5kit2", save_code = True, save_checkpoints = False),],
        ),
    param_space=config_param_space).fit()

## visualize

In [4]:
train_envs = 4
lr = 3e-4
lr_start = 3e-5
lr_end = 3e-6
lr_time = int(4e6)

config_param_space = {
    "env": "L5-CLE-V2",
    "framework": "torch",
    "num_gpus": 0,
    "num_workers": 8,
    "num_envs_per_worker": train_envs,
    "model": {
        "custom_model": "TorchAttentionModel4",
        "custom_model_config": {'cfg':cfg,
                                'freeze_actor': False,
                                'shared_feature_extractor': False,
                                },
    },
    
    # 'model' : {
    #         # "dim": 84,
    #         # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
    #         # "conv_activation": "relu",
    #         "post_fcnet_hiddens": [256],
    #         "post_fcnet_activation": "relu",
    #         "vf_share_layers": False,   
    # },
    
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    "lr": lr,
    'seed': 42,
    "lr_schedule": [
        [7e5, lr_start],
        [2e6, lr_end],
    ],
    'train_batch_size': 1024, # 8000 
    'sgd_minibatch_size': 512, #2048
    'num_sgd_iter': 10,#16,
    'batch_mode': 'truncate_episodes',
    # "rollout_fragment_length": 32,
    'gamma': 0.8,    
}

In [5]:
from ray import tune
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
eval_env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg, 'train': False, 'return_info': True}
# env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
rollout_env = L5Env2(**eval_env_kwargs)
tune.register_env("L5-CLE-EVAL-V2", lambda config: L5Env2(**eval_env_kwargs))

In [6]:
from ray.rllib.algorithms.ppo import PPO
checkpoint_path = '/home/pronton/ray_results/debug_vector_ppo_separated_kin_hist322-04-2023_13-05-42/PPO/PPO_L5-CLE-V2_b6ba7_00000_0_2023-04-22_06-05-42/checkpoint_000410'
algo = PPO(config=config_param_space, env='L5-CLE-EVAL-V2')
algo.restore(checkpoint_path)


2023-04-22 07:22:45,848	WARNING deprecation.py:47 -- DeprecationWarning: `algo = Algorithm(env='L5-CLE-EVAL-V2', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('L5-CLE-EVAL-V2').build()` instead. This will raise an error in the future!
2023-04-22 07:22:45,871	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-04-22 07:23:10,961	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-04-22 07:23:11,509	INFO trainable.py:172 -- Trainable.setup took 25.641 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-04-22 07:23:11,511	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2023-04-22 07:23:11,522	WARNING algorithm_config.py:488 -- Cannot create PPOConfig from given `config_dict`! Property __stdo

## Unroll scenes

In [ ]:
from src.simulation.unrollGym import unroll
sim_outs = unroll(model, rollout_env, 5,cfg['gym_params']['max_val_scene_id'])

In [ ]:
# might change with different rasterizer
map_API = rollout_env.dataset.rasterizer.sem_rast.mapAPI

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        # print(vis_in)
        # break
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
# visualize_outputs(sim_outs, map_API)